In [1]:
from pymongo import MongoClient
import pandas as pd
import pymongo

In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies
data = db.data_companies_clean.find()

In [3]:
data_comp = pd.DataFrame(data)
display(data_comp)

,_id,alias_list,category_code,description,founded_year,main_office,total_money_raised
0,5d261c1f797d7972f3178601,,web,Life long learning content,2007,"{'type': 'Point', 'coordinates': [-118.487267,...",$21M
1,5d261c1f797d7972f3178602,,web,Online Community and Discussion,2007,"{'type': 'Point', 'coordinates': [-119.306607,...",$12.7M
2,5d261c1f797d7972f3178603,,web,Video guide for hotels,2007,"{'type': 'Point', 'coordinates': [2.350987, 48...",$13.8M
3,5d261c1f797d7972f3178604,,web,Social software applications,2007,"{'type': 'Point', 'coordinates': [-122.419204,...",$16M
4,5d261c1f797d7972f3178605,None,web,None,2007,"{'type': 'Point', 'coordinates': [-105.276843,...",$315k
5,5d261c1f797d7972f3178606,,web,social media monitoring & analytics,2007,"{'type': 'Point', 'coordinates': [-105.276843,...",$1.92M
6,5d261c1f797d7972f3178607,SynthaSite,web,Free Websites,2007,"{'type': 'Point', 'coordinates': [-122.3954751...",$25M
7,5d261c1f797d7972f3178608,,web,,2007,"{'type': 'Point', 'coordinates': [-122.3276962...",$1.15M
8,5d261c1f797d7972f3178609,,web,Social Network and Media sharing,2007,"{'type': 'Point', 'coordinates': [-79.420926, ...",$3.5M
9,5d261c1f797d7972f317860a,,web,,2007,"{'type': 'Point', 'coordinates': [-122.084356,...",$450k


In [ ]:
#{founded_year:{$gte: 2011}}

In [4]:
def findNear(geopoint, radio_max_meters=1000):
    return db.first_office.find({
        "oficina_principal": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })


In [ ]:
#https://docs.mongodb.com/manual/reference/operator/query/near/index.html
def findNear(geopoint, radio_max_meters=1000):
    return db.first_office.find({
        "oficina_principal": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })


# https://developers.google.com/maps/documentation/geocoding/intro
park_avenue_con_21_street = {
    "type":"Point",
    "coordinates":[-73.987308,40.738935]
}
radio_max_meters = 10000
num_offices = findNear(park_avenue_con_21_street, radio_max_meters).count()

print(f"Hay {num_offices} oficinas cerca a {radio_max_meters} metros")
print("La mas cercana es")
print(list(findNear(park_avenue_con_21_street, radio_max_meters).limit(1)))

In [3]:
founded_year = db.oficinas.find({
    'founded_year':{
        '$gte': 2000
    }
    }

SyntaxError: unexpected EOF while parsing (<ipython-input-3-6c3d1b223bfb>, line 5)